In [12]:
from upaths import vtdem_fn,ftdem_fn,edem_fn

In [13]:
import rasterio
import numpy as np
from rasterio.features import shapes
from shapely.geometry import Polygon, Point
from rasterio.transform import Affine
from rasterio.crs import CRS
import os


def load_raster(filepath):
    """
    Load a raster file.
    """
    return rasterio.open(filepath)


def extract_voids(dem):
    """
    Extract a mask of voids from the DEM where values are:
    1. nodata (metadata-specified nodata value)
    2. < -99 or > 1000
    """
    dem_data = dem.read(1)
    nodata_value = dem.nodata

    # Create a binary mask of voids
    voids = np.zeros(dem_data.shape, dtype=np.uint8)
    voids[(dem_data == nodata_value) | (dem_data < -99) | (dem_data > 1000)] = 1

    return voids


def find_large_polygons(voids_mask, transform, min_area):
    """
    Identify large polygons of void areas in the voids mask.
    """
    huge_polys = []
    for poly, value in shapes(voids_mask, connectivity=8, transform=transform):
        if value == 1:  # Only consider void areas
            shp_poly = Polygon(poly['coordinates'][0])
            if shp_poly.area > min_area:
                huge_polys.append(shp_poly)
    return huge_polys


def create_grid_from_dem(dem):
    """
    Create X and Y coordinate arrays based on the DEM bounds and resolution.
    """
    bounds = dem.bounds  # Get the bounds of the DEM
    res = dem.transform[0]  # Pixel size from the transform
    x_min, x_max, y_min, y_max = bounds.left, bounds.right, bounds.bottom, bounds.top
    x_array = np.arange(x_min, x_max, res)
    y_array = np.arange(y_min, y_max, res)
    return x_array, y_array, res


def estimate_min_area(dem):
    """
    Estimate a minimum polygon area threshold as a fraction of the total DEM area.
    """
    total_area = (dem.bounds.right - dem.bounds.left) * (dem.bounds.top - dem.bounds.bottom)
    min_area = 0.0001 * total_area  # Use 0.01% of total area as a threshold
    return min_area


def interpolate_voids(dem2_raster, dem1_raster, void_polys, x_array, y_array):
    """
    Interpolates DEM2 voids using DEM1 values.
    """
    nrow = len(y_array)
    ncol = len(x_array)
    dem2_nodata = dem2_raster.nodata
    topo_filled = np.zeros((nrow, ncol), dtype=np.float32)

    for row in range(nrow):
        for col in range(ncol):
            col_x = x_array[col]
            row_y = y_array[row]
            pixel = Point(col_x, row_y)

            if any(poly.contains(pixel) for poly in void_polys):
                # Sample DEM2
                dem2_value = list(dem2_raster.sample([(col_x, row_y)]))[0][0]
                # Use DEM2 value if valid, otherwise propagate last valid value
                topo_filled[row, col] = (
                    topo_filled[row, col - 1] if dem2_value == dem2_nodata else dem2_value
                )
            else:
                # Sample DEM1
                dem1_value = list(dem1_raster.sample([(col_x, row_y)]))[0][0]
                topo_filled[row, col] = dem1_value

    return topo_filled


def save_raster(filepath, data, transform, crs, dtype):
    """
    Save data to a raster file.
    """
    with rasterio.open(
        filepath,
        'w',
        driver='GTiff',
        height=data.shape[0],
        width=data.shape[1],
        count=1,
        dtype=dtype,
        crs=crs,
        transform=transform,
    ) as dst:
        dst.write(data, 1)


def main(dem2_fpath, dem1_fpath, output_fpath):
    """
    Main workflow to fill voids in DEM2 using DEM1.
    """
    # Load DEMs
    dem2_raster = load_raster(dem2_fpath)
    dem1_raster = load_raster(dem1_fpath)
    print('Loaded DEMs.')

    # Extract voids from DEM2
    voids_mask = extract_voids(dem2_raster)
    print('Extracted voids from DEM2.')
    print(f"Number of void pixels: {np.sum(voids_mask)}")

    # Automatically determine resolution and min_area
    x_array, y_array, res = create_grid_from_dem(dem2_raster)
    min_area = estimate_min_area(dem2_raster)
    print(f'Determined resolution: {res}, Minimum area threshold: {min_area}')

    # Find large void polygons
    void_polys = find_large_polygons(voids_mask, dem2_raster.transform, min_area)
    print(f'Number of large void polygons: {len(void_polys)}')

    # Handle case with no polygons
    if not void_polys:
        print("No large void polygons found. Exiting.")
        return

    # Interpolate voids in DEM2 using DEM1
    topo_filled = interpolate_voids(dem2_raster, dem1_raster, void_polys, x_array, y_array)
    print('Interpolated voids.')

    # Save filled DEM
    transform = Affine.translation(x_array[0] - res / 2, y_array[0] - res / 2) * Affine.scale(res, res)
    crs = dem2_raster.crs  # Use the CRS of DEM2
    os.remove(output_fpath) if os.path.exists(output_fpath) else None
    save_raster(output_fpath, topo_filled, transform, crs, topo_filled.dtype)
    print('Saved filled DEM.')


# Example usage
# main("dem2.tif", "dem1.tif", "filled_dem2.tif")


In [15]:
import rasterio
import numpy as np
from rasterio.features import shapes
from shapely.geometry import Polygon, Point
from rasterio.transform import Affine
from rasterio.crs import CRS
import os


def load_raster(filepath):
    """
    Load a raster file.
    """
    return rasterio.open(filepath)


def extract_voids(dem):
    """
    Extract a mask of voids from the DEM where values are:
    1. nodata (metadata-specified nodata value)
    2. < -99 or > 1000
    """
    dem_data = dem.read(1)
    nodata_value = dem.nodata

    # Create a binary mask of voids
    voids = np.zeros(dem_data.shape, dtype=np.uint8)
    voids[(dem_data == nodata_value) | (dem_data < -99) | (dem_data > 1000)] = 1

    return voids


def find_void_polygons(voids_mask, transform, min_area):
    """
    Identify void polygons in the mask that are larger than the minimum area threshold.
    """
    void_polys = []
    for poly, value in shapes(voids_mask, connectivity=8, transform=transform):
        if value == 1:  # Only consider void areas
            shp_poly = Polygon(poly['coordinates'][0])
            if shp_poly.area > min_area:
                void_polys.append(shp_poly)
    return void_polys


def create_grid_from_dem(dem):
    """
    Create X and Y coordinate arrays based on the DEM bounds and resolution.
    """
    bounds = dem.bounds  # Get the bounds of the DEM
    res = dem.transform[0]  # Pixel size from the transform
    x_min, x_max, y_min, y_max = bounds.left, bounds.right, bounds.bottom, bounds.top
    x_array = np.arange(x_min, x_max, res)
    y_array = np.arange(y_min, y_max, res)
    return x_array, y_array, res


def estimate_min_area(dem):
    """
    Estimate a minimum polygon area threshold as a fraction of the total DEM area.
    """
    total_area = (dem.bounds.right - dem.bounds.left) * (dem.bounds.top - dem.bounds.bottom)
    min_area = 0.0001 * total_area  # Use 0.01% of total area as a threshold
    return min_area


def interpolate_voids(dem2_raster, dem1_raster, void_polys, x_array, y_array):
    """
    Interpolates DEM2 voids using DEM1 values.
    """
    nrow = len(y_array)
    ncol = len(x_array)
    dem2_nodata = dem2_raster.nodata
    topo_filled = np.zeros((nrow, ncol), dtype=np.float32)

    for row in range(nrow):
        for col in range(ncol):
            col_x = x_array[col]
            row_y = y_array[row]
            pixel = Point(col_x, row_y)

            if any(poly.contains(pixel) for poly in void_polys):
                # Sample DEM2
                dem2_value = list(dem2_raster.sample([(col_x, row_y)]))[0][0]
                # Use DEM2 value if valid, otherwise propagate last valid value
                topo_filled[row, col] = (
                    topo_filled[row, col - 1] if dem2_value == dem2_nodata else dem2_value
                )
            else:
                # Sample DEM1
                dem1_value = list(dem1_raster.sample([(col_x, row_y)]))[0][0]
                topo_filled[row, col] = dem1_value

    return topo_filled


def save_raster(filepath, data, transform, crs, dtype):
    """
    Save data to a raster file.
    """
    with rasterio.open(
        filepath,
        'w',
        driver='GTiff',
        height=data.shape[0],
        width=data.shape[1],
        count=1,
        dtype=dtype,
        crs=crs,
        transform=transform,
    ) as dst:
        dst.write(data, 1)


def main(dem2_fpath, dem1_fpath, output_fpath):
    """
    Main workflow to fill voids in DEM2 using DEM1.
    """
    # Load DEMs
    dem2_raster = load_raster(dem2_fpath)
    dem1_raster = load_raster(dem1_fpath)
    print('Loaded DEMs.')

    # Extract voids from DEM2
    voids_mask = extract_voids(dem2_raster)
    print('Extracted voids from DEM2.')
    print(f"Number of void pixels: {np.sum(voids_mask)}")

    # Automatically determine resolution and min_area
    x_array, y_array, res = create_grid_from_dem(dem2_raster)
    min_area = estimate_min_area(dem2_raster)
    print(f'Determined resolution: {res}, Minimum area threshold: {min_area}')

    # Find void polygons for large, medium, and small holes
    # Large polygons first, then medium, small, and finally fill any voids.
    void_polys = []
    thresholds = [min_area, min_area * 0.5, min_area * 0.1]  # Gradually lower the threshold
    for threshold in thresholds:
        current_polys = find_void_polygons(voids_mask, dem2_raster.transform, threshold)
        void_polys.extend(current_polys)
        print(f"Found {len(current_polys)} void polygons at threshold {threshold}.")
        if len(void_polys) > 0:
            break  # Stop if we found any void polygons to fill

    # Handle case with no polygons
    if not void_polys:
        print("No large void polygons found. Filling all voids.")
        void_polys = find_void_polygons(voids_mask, dem2_raster.transform, 0)  # Fill all voids

    # Interpolate voids in DEM2 using DEM1
    topo_filled = interpolate_voids(dem2_raster, dem1_raster, void_polys, x_array, y_array)
    print('Interpolated voids.')

    # Save filled DEM
    transform = Affine.translation(x_array[0] - res / 2, y_array[0] - res / 2) * Affine.scale(res, res)
    crs = dem2_raster.crs  # Use the CRS of DEM2
    os.remove(output_fpath) if os.path.exists(output_fpath) else None
    save_raster(output_fpath, topo_filled, transform, crs, topo_filled.dtype)
    print('Saved filled DEM.')


# Example usage
# main("dem2.tif", "dem1.tif", "filled_dem2.tif")


In [16]:
main(vtdem_fn, edem_fn, ftdem_fn)

Loaded DEMs.
Extracted voids from DEM2.
Number of void pixels: 0
Determined resolution: 0.0008333333333333334, Minimum area threshold: 0.0001
Found 0 void polygons at threshold 0.0001.
Found 0 void polygons at threshold 5e-05.
Found 0 void polygons at threshold 1e-05.
No large void polygons found. Filling all voids.
Interpolated voids.
Saved filled DEM.


In [17]:
# this working well, but it's shifting the values to the right a bit
# add post processing to regrid the data into orignal bounds